In [ ]:
from flask import Flask, Response, stream_with_context
from ultralytics import YOLO
import cv2, numpy as np

# ——— CONFIG ———
STREAM_URL = "http://192.168.137.65/stream"  # ← your verified ESP32 URL
MODEL_PATH = "best.pt"

model = YOLO(MODEL_PATH)
app   = Flask(__name__)

def gen_frames():
    cap = cv2.VideoCapture(STREAM_URL)
    # retry until we get a connection
    while not cap.isOpened():
        cap.open(STREAM_URL)
    while True:
        ret, frame = cap.read()
        if not ret:
            continue
        # … do your YOLO drawing here …
        _, jpg = cv2.imencode('.jpg', frame)
        yield (b'--frame\r\n'
               b'Content-Type: image/jpeg\r\n\r\n' +
               jpg.tobytes() + b'\r\n')

@app.route('/stream')
def stream():
    return Response(
        stream_with_context(gen_frames()),
        mimetype='multipart/x-mixed-replace; boundary=frame'
    )

if __name__ == '__main__':
    # match the port you actually saw in your logs
    app.run(host='192.168.137.33', port=5001, threaded=True)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://192.168.137.33:5000
Press CTRL+C to quit
192.168.137.33 - - [30/May/2025 22:46:05] "GET / HTTP/1.1" 404 -


In [15]:
import cv2
import time
import os

STREAM_URL = "http://192.168.137.77/stream"  # set your ESP32 stream URL
OUTPUT_DIR = "captures"                     # where to save images
DURATION   = 30                             # total seconds to capture
INTERVAL   = 1.0                            # seconds between frames

os.makedirs(OUTPUT_DIR, exist_ok=True)

cap = cv2.VideoCapture(STREAM_URL)
if not cap.isOpened():
    raise RuntimeError(f"Cannot open stream at {STREAM_URL}")

start_time = time.time()
count = 0

try:
    while time.time() - start_time < DURATION:
        ret, frame = cap.read()
        if not ret:
            print("Frame grab failed; retrying…")
            continue

        filename = os.path.join(OUTPUT_DIR, f"capture_{count:02d}.jpg")
        cv2.imwrite(filename, frame)
        print(f"[{count+1}/30] Saved {filename}")

        count += 1
        time.sleep(INTERVAL)

finally:
    cap.release()
    cv2.destroyAllWindows()
    print("Done capturing 30 frames.")


[1/30] Saved captures\capture_00.jpg
[2/30] Saved captures\capture_01.jpg
[3/30] Saved captures\capture_02.jpg
[4/30] Saved captures\capture_03.jpg
[5/30] Saved captures\capture_04.jpg
[6/30] Saved captures\capture_05.jpg
[7/30] Saved captures\capture_06.jpg
[8/30] Saved captures\capture_07.jpg
[9/30] Saved captures\capture_08.jpg
[10/30] Saved captures\capture_09.jpg
[11/30] Saved captures\capture_10.jpg
[12/30] Saved captures\capture_11.jpg
[13/30] Saved captures\capture_12.jpg
[14/30] Saved captures\capture_13.jpg
[15/30] Saved captures\capture_14.jpg
[16/30] Saved captures\capture_15.jpg
[17/30] Saved captures\capture_16.jpg
[18/30] Saved captures\capture_17.jpg
[19/30] Saved captures\capture_18.jpg
[20/30] Saved captures\capture_19.jpg
[21/30] Saved captures\capture_20.jpg
[22/30] Saved captures\capture_21.jpg
[23/30] Saved captures\capture_22.jpg
[24/30] Saved captures\capture_23.jpg
[25/30] Saved captures\capture_24.jpg
[26/30] Saved captures\capture_25.jpg
[27/30] Saved capture

In [ ]:
from ultralytics import YOLO

model = YOLO("best.pt")

res = model.predict(source="captures", save=True, conf=0.1)




image 1/30 c:\Users\bobe\Documents\Bobe\CD\captures\capture_00.jpg: 480x640 1 No Fall, 40.3ms
image 2/30 c:\Users\bobe\Documents\Bobe\CD\captures\capture_01.jpg: 480x640 (no detections), 37.4ms
image 3/30 c:\Users\bobe\Documents\Bobe\CD\captures\capture_02.jpg: 480x640 (no detections), 35.0ms
image 4/30 c:\Users\bobe\Documents\Bobe\CD\captures\capture_03.jpg: 480x640 (no detections), 34.8ms
image 5/30 c:\Users\bobe\Documents\Bobe\CD\captures\capture_04.jpg: 480x640 (no detections), 38.5ms
image 6/30 c:\Users\bobe\Documents\Bobe\CD\captures\capture_05.jpg: 480x640 (no detections), 40.9ms
image 7/30 c:\Users\bobe\Documents\Bobe\CD\captures\capture_06.jpg: 480x640 (no detections), 40.5ms
image 8/30 c:\Users\bobe\Documents\Bobe\CD\captures\capture_07.jpg: 480x640 (no detections), 37.9ms
image 9/30 c:\Users\bobe\Documents\Bobe\CD\captures\capture_08.jpg: 480x640 (no detections), 37.2ms
image 10/30 c:\Users\bobe\Documents\Bobe\CD\captures\capture_09.jpg: 480x640 (no detections), 32.9ms
imag

In [9]:
from ultralytics import YOLO
import paho.mqtt.client as mqtt
import os

# ——— CONFIG ———
MODEL_PATH  = "best.pt"
SOURCE_DIR  = "captures"
CONF_THRESH = 0.1

# MQTT settings
MQTT_BROKER = "192.168.137.216"
MQTT_PORT   = 1883
MQTT_TOPIC  = "home/tulung_jatuh"

# ——— SETUP MQTT ———
client = mqtt.Client()
client.username_pw_set("mqtt_user", "mqtt1234")
client.connect(MQTT_BROKER, MQTT_PORT, keepalive=60)
client.loop_start()

# ——— RUN PREDICTIONS ———
model = YOLO(MODEL_PATH)
results = model.predict(source=SOURCE_DIR, save=True, conf=CONF_THRESH)

# ——— PROCESS & PUBLISH ———
for res in results:
    # res.boxes is a list of Box objects for this image
    fall_detected = any(
        model.names[int(box.cls[0])].lower() == "fall"
        for box in res.boxes
    )
    payload = "P Jatoh" if fall_detected else "Belom Jatoh jir"
    client.publish(MQTT_TOPIC, payload, qos=1, retain=True)

    # extract just the filename from the path string
    filename = os.path.basename(res.path)
    print(f"[{filename}] fall_detected={fall_detected} → published {payload}")

# ——— CLEANUP ———
client.loop_stop()
client.disconnect()



image 1/30 c:\Users\bobe\Documents\Bobe\CD\captures\capture_00.jpg: 480x640 1 No Fall, 50.3ms
image 2/30 c:\Users\bobe\Documents\Bobe\CD\captures\capture_01.jpg: 480x640 (no detections), 42.6ms


C:\Users\bobe\AppData\Local\Temp\ipykernel_21264\2120889444.py:16: DeprecationWarning: Callback API version 1 is deprecated, update to latest version
  client = mqtt.Client()


image 3/30 c:\Users\bobe\Documents\Bobe\CD\captures\capture_02.jpg: 480x640 (no detections), 40.9ms
image 4/30 c:\Users\bobe\Documents\Bobe\CD\captures\capture_03.jpg: 480x640 (no detections), 51.0ms
image 5/30 c:\Users\bobe\Documents\Bobe\CD\captures\capture_04.jpg: 480x640 (no detections), 37.7ms
image 6/30 c:\Users\bobe\Documents\Bobe\CD\captures\capture_05.jpg: 480x640 (no detections), 35.4ms
image 7/30 c:\Users\bobe\Documents\Bobe\CD\captures\capture_06.jpg: 480x640 (no detections), 36.7ms
image 8/30 c:\Users\bobe\Documents\Bobe\CD\captures\capture_07.jpg: 480x640 (no detections), 38.8ms
image 9/30 c:\Users\bobe\Documents\Bobe\CD\captures\capture_08.jpg: 480x640 (no detections), 36.7ms
image 10/30 c:\Users\bobe\Documents\Bobe\CD\captures\capture_09.jpg: 480x640 (no detections), 37.8ms
image 11/30 c:\Users\bobe\Documents\Bobe\CD\captures\capture_10.jpg: 480x640 (no detections), 40.5ms
image 12/30 c:\Users\bobe\Documents\Bobe\CD\captures\capture_11.jpg: 480x640 (no detections), 36.5

<MQTTErrorCode.MQTT_ERR_SUCCESS: 0>